In [15]:
from pymodulon.io import save_to_json, load_json_model
import pandas as pd

In [8]:
ica_data = load_json_model('../data/temp/p1k_starve_series.json')

In [13]:
ica_data.sample_table.to_csv('./delete_me.csv')

In [17]:
log_tpm_df = pd.read_csv('../data/precise_1.0/log_tpm.csv', index_col = 0)
starve_log_tpm = pd.read_csv('../data/validation_data_sets/stationary_phase/project_log_tpm_qc.csv', index_col = 0)
to_blank_inds = list(set(log_tpm_df.index) - set(starve_log_tpm.index))
# need to create zero rows for missing values
zeros_data = {col : 0 for col in starve_log_tpm.columns}
zeros_df = pd.DataFrame(zeros_data, index = to_blank_inds)
starve_log_tpm = pd.concat([starve_log_tpm, zeros_df])
starve_log_tpm = starve_log_tpm.loc[log_tpm_df.index]
log_tpm_df = pd.concat([starve_log_tpm, log_tpm_df], axis = 1)

In [25]:
M_df = pd.read_csv('../data/precise_1.0/M.csv', index_col = 0)
iM_table = pd.read_csv('../data/precise_1.0/iM_table.csv', index_col = 0)
M_df = M_df.rename(columns = {str(index) : row['name'] for index, row in iM_table.iterrows()})

In [26]:
M_df

,YieP,GlpR,YgbI,efeU-repair,ydcI-KO,BW25113,proVWX,DhaR/Mlc,ArgR,YneJ,...,RpoS,membrane,duplication-1,uncharacterized-3,ExuR/FucR,FecI,iron-related,RbsR,sgrT,insertion
b0002,0.072693,-0.006320,-0.038374,-0.010045,-0.004345,-0.005599,0.009043,-0.005072,-0.012186,-0.013543,...,-0.010152,-0.012124,-0.012027,-0.002123,-0.000036,-0.013667,-0.018883,0.004408,-0.006315,-0.001780
b0003,0.015975,-0.009969,-0.024571,-0.006775,-0.003583,0.006680,-0.010841,-0.012620,-0.003685,0.003512,...,0.004702,0.004402,0.001182,-0.004415,-0.001514,-0.005052,0.015502,0.000860,-0.022438,-0.020087
b0004,0.015319,-0.003808,-0.027018,-0.004455,-0.008418,0.003245,-0.016494,-0.013818,-0.008104,0.002144,...,0.003507,0.005342,0.003759,0.000700,0.001859,-0.002344,0.016263,0.002486,-0.016565,-0.014142
b0005,0.008229,-0.000685,-0.005215,-0.002029,-0.028069,0.014748,-0.019240,-0.002403,-0.008507,-0.019092,...,-0.006856,0.010676,0.008874,0.018497,0.000700,-0.013267,0.015265,0.016886,-0.015319,-0.020727
b0006,-0.004941,-0.001849,-0.002326,0.006075,0.001607,0.009678,0.001661,0.003754,-0.000152,-0.001013,...,-0.014761,0.005071,0.001496,-0.002506,-0.002819,0.005800,0.003975,-0.010976,-0.008756,0.006169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b4688,-0.030311,0.008120,0.011910,0.001713,-0.005233,0.014935,0.019655,-0.025658,0.011851,-0.020593,...,-0.016448,0.009670,0.008979,0.003438,-0.019520,-0.067713,-0.021288,0.012194,0.028681,-0.025933
b4693,0.027739,0.011624,-0.013381,0.013677,0.005577,-0.008841,0.001735,0.007473,-0.007097,0.031408,...,-0.007476,-0.003357,0.001309,-0.014741,0.004931,-0.007483,-0.017265,0.000494,-0.000197,0.042749
b4696_1,0.009761,0.005718,-0.001005,-0.004127,-0.000752,0.017119,0.006490,0.001626,-0.002876,0.010560,...,-0.007016,0.001603,0.000678,0.007108,0.000750,0.004399,-0.010742,-0.004032,-0.004957,-0.003783
b4696_2,-0.001164,0.007835,0.001547,0.002629,-0.009831,0.001307,-0.009274,0.006053,-0.003772,0.010440,...,-0.003376,0.000974,0.003979,0.030489,0.007771,0.009035,-0.008468,-0.002449,0.004692,0.007504


In [19]:
log_tpm_df

,control__wt_glc__1,control__wt_glc__2,fur__wt_dpd__1,fur__wt_dpd__2,fur__wt_fe__1,fur__wt_fe__2,fur__delfur_dpd__1,fur__delfur_dpd__2,fur__delfur_fe2__1,fur__delfur_fe2__2,...,efeU__menFentC_ale29__1,efeU__menFentC_ale29__2,efeU__menFentC_ale30__1,efeU__menFentC_ale30__2,efeU__menFentCubiC_ale36__1,efeU__menFentCubiC_ale36__2,efeU__menFentCubiC_ale37__1,efeU__menFentCubiC_ale37__2,efeU__menFentCubiC_ale38__1,efeU__menFentCubiC_ale38__2
b0002,11.006653,11.130196,11.704951,11.888217,11.064810,10.779071,9.976401,10.291135,11.229767,11.214065,...,10.271327,10.276565,11.148538,11.170578,11.676604,11.726097,11.881529,11.923237,11.496410,11.552762
b0003,10.415654,10.523138,11.423835,11.803781,10.776984,10.597810,9.596833,10.191503,10.897938,10.861157,...,10.160291,10.116861,10.314322,10.392251,10.916426,10.909277,11.023924,11.038426,10.624301,10.764195
b0004,10.594871,10.725062,10.457270,10.779162,10.394971,10.113950,8.741617,9.082231,10.185151,10.164655,...,10.475069,10.434352,10.679541,10.723953,11.143310,11.112721,11.184795,11.241845,10.953206,11.001006
b0005,6.316102,6.258499,5.422129,5.336121,6.716069,6.410864,4.626949,4.756153,6.527653,6.136168,...,5.979079,5.705586,6.306120,6.291340,5.058537,4.835550,5.448097,5.757951,5.873964,5.808618
b0006,6.915771,6.897597,6.775645,6.782605,6.761813,6.816532,6.686767,6.860036,6.862147,6.817480,...,8.371287,8.322390,8.137515,8.071837,7.354131,7.365536,7.328101,7.314761,8.057750,8.105213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b4688,2.473165,2.995816,1.308909,0.000000,2.916383,3.248885,0.790543,0.741789,2.800527,2.039165,...,1.849193,2.272005,0.000000,1.283342,1.355421,1.167070,1.734880,1.007913,0.000000,1.545422
b4693,0.868914,1.426733,2.509185,2.168134,1.755932,2.136364,3.706239,3.290548,4.268691,4.252711,...,0.772861,2.004397,0.000000,0.333735,3.202295,3.001444,3.105540,3.091406,3.380939,3.171579
b4696_1,2.646501,2.545448,3.762411,3.639348,2.064534,2.014348,3.510030,3.327140,2.468705,2.431928,...,2.857578,2.874401,2.797063,2.578195,2.734153,2.718261,3.100377,3.021188,3.225358,3.401920
b4696_2,3.084724,3.148030,3.901950,3.997730,2.639106,2.200281,3.953980,3.917770,3.044666,3.115836,...,2.617006,3.515160,3.212986,3.012930,2.597279,3.731740,3.460336,3.696665,3.483282,3.818985


In [1]:
import os

In [5]:
folders = [val for val in os.listdir('../data/saved_run_results') if 'run' in val]
if len(folders) == 0:
    run_ct = 1
else:
    run_ct = max([int(val.split('_')[1]) for val in folders]) + 1
new_run_folder = 

In [3]:
ls ../data

act_inh_pkl/                precise_1k/
cAct_cInh_vals/             saved_flags.csv
GAMS_output/                saved_mRNA_ratios_MA_vals/
gene_grid_constants/        saved_ptsG_predictions.pkl
gene_to_grid_constants.pkl  saved_run_results/
pickle_dfs/                 save_for_GAMs/
precise_1.0/                validation_data_sets/


In [17]:
import dill as pickle
import pandas as pd

In [18]:
ls ../data/act_inh_pkl

231003-b0723-greedy.pkl  231003-b1101-greedy.pkl  231003-b2239-greedy.pkl
231003-b0723.pkl         231003-b1101.pkl         231003-b2239.pkl


In [25]:
pickle_in = open('../data/act_inh_pkl/231003-b2239-greedy.pkl', 'rb')
a = pickle.load(pickle_in)

In [26]:
for val in a[0]:
    print(val)

3.630780547701014
2.884031503126606
1.7868012422392947
8.31763771102671
0.02290299101575983
0.32274567015748645
8.766983113552968
4.36515832240166
3.311311214825911
1.4454397707459274
5.011872336272724
1.4454397707459274
0.004931594317965625
2.423423603095939
2.4975121157159146
2.3988329190194904
9.120108393559097
4.168693834703354
1.0471285480508996
1.9952623149688795
1.096478196143185
0.07385234756199617
8.709635899560807
1.9952623149688795
1.0471285480508996
1.318256738556407
1.096478196143185
8.571910962764187
0.00407336746012933
0.0022547594561292965
0.20602318628389196
0.0012809129386032127
8.709635899560807
9.052135781779484
2.2194078173537797
2.850924674043761
1.390342388363182
2.0892961308540396
2.290867652767773
2.129391655928126
1.6595869074375607
0.6094688623440997
0.30150810794346394
2.884031503126606
0.0792939281823341
0.23070761952640131
0.0746842968610376
0.07511780454562579
1.9054607179632472
0.21763556090210393
0.009746605341139183
0.009993357967132988
0.0089877547690

In [10]:
a[1]

[158.48931924611142,
 115.10260045756357,
 467.72430577572993,
 367.20595561981486,
 131.56826064833706,
 130.69082724521732,
 398.1071705534973,
 364.5040313138397,
 304.9597388773301,
 177.82794100389228,
 161.63941866040753,
 159.8414720813878,
 83.4414405634976,
 158.66437984592878,
 160.3061133241512,
 158.48931924611142,
 163.08844253608459,
 141.47921883779551,
 177.82794100389228,
 251.18864315095823,
 177.82794100389228,
 155.85897002509537,
 354.81338923357566,
 253.3998178395233,
 199.52623149688807,
 187.87157649207367,
 188.574778498691,
 195.286012302152,
 0.28183829312644537,
 63.095734448019364,
 141.25375446227554,
 1.0,
 255.80017843862316,
 420.6906174144198,
 179.45618895484083,
 160.76100389622357,
 158.48931924611142,
 186.8513149655894,
 138.07200998509828,
 172.508936760998,
 178.54722404619707,
 137.92873585919483,
 132.3606786017897,
 140.14789569838922,
 112.28132146803696,
 124.30716203302838,
 141.25375446227554,
 138.69386318339315,
 181.79741279213636,
 1

In [11]:
a[2]

['control__wt_glc__1',
 'control__wt_glc__2',
 'fur__wt_dpd__1',
 'fur__wt_dpd__2',
 'fur__wt_fe__1',
 'fur__wt_fe__2',
 'fur__delfur_dpd__1',
 'fur__delfur_dpd__2',
 'fur__delfur_fe2__1',
 'fur__delfur_fe2__2',
 'acid__wt_ph5__1',
 'acid__wt_ph5__2',
 'acid__delgadx_ph5__1',
 'acid__delgadx_ph5__2',
 'acid__delgadw_ph5__1',
 'acid__delgadw_ph5__2',
 'acid__delgade_ph5__1',
 'acid__delgade_ph5__2',
 'oxidative__wt_pq__1',
 'oxidative__wt_pq__2',
 'oxidative__deloxyr_pq__1',
 'oxidative__deloxyr_pq__2',
 'oxidative__delsoxr_pq__1',
 'oxidative__delsoxr_pq__2',
 'oxidative__delsoxs_pq__1',
 'oxidative__delsoxs_pq__2',
 'nac_ntrc__wt_csn__1',
 'nac_ntrc__wt_csn__2',
 'nac_ntrc__wt_cytd__1',
 'nac_ntrc__wt_cytd__2',
 'nac_ntrc__wt_gln__1',
 'nac_ntrc__wt_gln__2',
 'nac_ntrc__bw_delnac_csn__1',
 'nac_ntrc__bw_delnac_csn__2',
 'nac_ntrc__bw_delnac_cytd__1',
 'nac_ntrc__bw_delnac_cytd__2',
 'nac_ntrc__bw_delnac_gln__1',
 'nac_ntrc__bw_delnac_gln__2',
 'nac_ntrc__bw_delnac_nh4__1',
 'nac_ntrc_

In [12]:
a[3]

(0.8952992021507751, -0.9399409588017691)

In [13]:
a[4]

{'KdRNAP': 1e-05,
 'KdRNAPCrp': 3.516641004113411e-07,
 'KeqOpening': 0.4524289955987184,
 'RNAP': 1e-06}

In [14]:
a[5]

{'seed': 42,
 'n_gen': 1000,
 'n_ind': 100,
 'n_iter': 10,
 'mu': 100,
 'lambda_': 100,
 'cx_ind': 1,
 'cx_gene': 0.6,
 'mt_ind': 0,
 'mt_gene': 0.1}

In [15]:
a[6]

IndexError: list index out of range